# Genetic Algorithm for Knapsack Problem
This notebook compares a custom Genetic Algorithm (GA), DEAP GA, and brute-force search for solving a small Knapsack problem.

In [1]:
import random
import time
from deap import base, creator, tools, algorithms

## Step 1: Define knapsack problem
We define the items (weight, value) and the maximum weight allowed in the knapsack.

In [2]:
# Items: (weight, value)
items = [(3, 60), (1, 4), (30, 12), (10, 70), (2, 40), (1, 2), (7, 60), (13, 30)]
MAX_WEIGHT = 60
num_items = len(items)

# GA parameters
MUTATION_RATE = 0.01
POP_SIZE = 50
CROSSOVER_RATE = 0.8
NUM_GENERATIONS = 100
NUM_PARENTS = int(POP_SIZE * 0.5)

## Step 2: Define custom GA functions
Functions include creating individuals, population, fitness evaluation, selection, crossover, mutation, and generating new populations.

In [3]:
def create_individual(num_items):
    return [random.randint(0, 1) for _ in range(num_items)]

def create_population(pop_size, num_items):
    return [create_individual(num_items) for _ in range(pop_size)]

def fitness(individual, items, max_weight):
    total_weight = 0
    total_value = 0
    for i in range(len(individual)):
        if individual[i] == 1:
            total_weight += items[i][0]
            total_value += items[i][1]
    return total_value if total_weight <= max_weight else 0

def select_parents(population, items, max_weight, num_parents):
    evaluated = [(fitness(ind, items, max_weight), ind) for ind in population]
    evaluated.sort(key=lambda x: x[0], reverse=True)
    selected = [ind for (score, ind) in evaluated[:num_parents]]
    
    print("\nSelected parents (best individuals):")
    for i, p in enumerate(selected):
        print(f"Parent {i+1}: {p} | Fitness = {fitness(p, items, max_weight)}")
    return selected

def crossover(parent1, parent2):
    size = len(parent1)
    if size <= 1:
        return parent1[:], parent2[:]
    point = random.randint(1, size - 1)
    child1 = parent1[:point] + parent2[point:]
    child2 = parent2[:point] + parent1[point:]
    print(f"\nCrossover:\n  {parent1}\n  {parent2}\n  point={point}\n  Children={child1}, {child2}")
    return child1, child2

def mutate(individual, mutation_rate):
    mutated = individual[:]
    for i in range(len(mutated)):
        if random.random() < mutation_rate:
            mutated[i] = 1 - mutated[i]
    if individual != mutated:
        print(f"Mutation: {individual} -> {mutated}")
    return mutated

def new_generation(parents, pop_size, mutation_rate):
    new_pop = parents[:]
    while len(new_pop) < pop_size:
        p1 = random.choice(parents)
        p2 = random.choice(parents)
        c1, c2 = crossover(p1, p2)
        c1 = mutate(c1, mutation_rate)
        c2 = mutate(c2, mutation_rate)
        new_pop.append(c1)
        if len(new_pop) < pop_size:
            new_pop.append(c2)
    print("\nNew generation:")
    for i, ind in enumerate(new_pop):
        print(f"Individual {i+1}: {ind} | Fitness = {fitness(ind, items, MAX_WEIGHT)}")
    return new_pop

## Step 3: Run custom GA
Run the GA over multiple generations and track the best solution.

In [4]:
def run_custom_ga(seed=42):
    random.seed(seed)
    population = create_population(POP_SIZE, num_items)
    best_score = 0
    best_individual = None
    t0 = time.time()
    
    for generation in range(NUM_GENERATIONS):
        parents = select_parents(population, items, MAX_WEIGHT, NUM_PARENTS)
        score_best = fitness(parents[0], items, MAX_WEIGHT)
        if score_best > best_score:
            best_score = score_best
            best_individual = parents[0][:]
        population = new_generation(parents, POP_SIZE, MUTATION_RATE)
    
    t1 = time.time()
    return {"value": best_score, "ind": best_individual, "time": t1 - t0}

## Step 4: DEAP GA
Setup DEAP genetic algorithm for the knapsack problem.

In [5]:
creator.create("FitnessMax", base.Fitness, weights=(1.0,))
creator.create("Individual", list, fitness=creator.FitnessMax)

toolbox = base.Toolbox()
toolbox.register("attr_bool", random.randint, 0, 1)
toolbox.register("individual", tools.initRepeat, creator.Individual, toolbox.attr_bool, n=len(items))
toolbox.register("population", tools.initRepeat, list, toolbox.individual)

def eval_knapsack(ind):
    total_weight = sum(items[i][0] for i in range(len(ind)) if ind[i])
    total_value = sum(items[i][1] for i in range(len(ind)) if ind[i])
    return (total_value,) if total_weight <= MAX_WEIGHT else (0,)

toolbox.register("evaluate", eval_knapsack)
toolbox.register("mate", tools.cxTwoPoint)
toolbox.register("mutate", tools.mutFlipBit, indpb=MUTATION_RATE)
toolbox.register("select", tools.selTournament, tournsize=3)

def run_deap_ga(seed=42):
    random.seed(seed)
    pop = toolbox.population(n=POP_SIZE)
    t0 = time.time()
    pop, logbook = algorithms.eaSimple(pop, toolbox,
                                       cxpb=CROSSOVER_RATE,
                                       mutpb=MUTATION_RATE,
                                       ngen=NUM_GENERATIONS,
                                       stats=None,
                                       verbose=False)
    t1 = time.time()
    best = tools.selBest(pop, k=1)[0]
    return {"value": best.fitness.values[0], "ind": list(best), "time": t1 - t0}

## Step 5: Brute force exact solution
Compute all possible combinations to find the optimal solution.

In [6]:
def brute_force_opt(objects, max_weight):
    best_val = -10**9
    best_ind = None
    for mask in range(1<<len(objects)):
        ind = [(mask>>i)&1 for i in range(len(objects))]
        w = sum(ind[i]*objects[i][0] for i in range(len(objects)))
        v = sum(ind[i]*objects[i][1] for i in range(len(objects)))
        if w <= max_weight and v > best_val:
            best_val = v
            best_ind = ind[:]
    return best_val, best_ind, sum(best_ind[i]*objects[i][0] for i in range(len(objects)))

## Step 6: Compare custom GA, DEAP GA, and brute force
Run all methods and compare the results.

In [7]:
if __name__ == "__main__":
    print("\n" + "="*60)
    print("COMPARISON: CUSTOM GA vs DEAP GA vs BRUTE FORCE")
    print("="*60)
    print(f"Items ({len(items)}), MAX_WEIGHT = {MAX_WEIGHT}")
    print(f"Parameters: pop={POP_SIZE}, gens={NUM_GENERATIONS}, cx={CROSSOVER_RATE}, mut={MUTATION_RATE}")
    print("="*60 + "\n")

    res_custom = run_custom_ga(seed=42)
    print("CUSTOM GA:")
    print(f"  Value: {res_custom['value']}")
    print(f"  Knapsack (ind): {res_custom['ind']}")
    print(f"  Time (s): {res_custom['time']:.4f}\n")

    res_deap = run_deap_ga(seed=42)
    print("DEAP GA:")
    print(f"  Value: {res_deap['value']}")
    print(f"  Knapsack (ind): {res_deap['ind']}")
    print(f"  Time (s): {res_deap['time']:.4f}\n")

    opt_value, opt_ind, opt_weight = brute_force_opt(items, MAX_WEIGHT)
    print("BRUTE FORCE (exact):")
    print(f"  Optimal value: {opt_value}")
    print(f"  Optimal knapsack: {opt_ind}")
    print(f"  Optimal weight: {opt_weight}\n")

    print("="*60)
    def show_compare(name, res):
        val = res['value']
        ind = res['ind']
        ok = "OK" if val == opt_value else "NOT"
        print(f"{name:10s} -> value={val:3d} | optimal={ok} | ind={ind}")
    show_compare("CUSTOM GA", res_custom)
    show_compare("DEAP GA", {"value": int(res_deap['value']), "ind": res_deap['ind']})
    print("="*60 + "\n")


COMPARISON: CUSTOM GA vs DEAP GA vs BRUTE FORCE
Items (8), MAX_WEIGHT = 60
Parameters: pop=50, gens=100, cx=0.8, mut=0.01


Selected parents (best individuals):
Parent 1: [1, 1, 1, 1, 1, 0, 1, 0] | Fitness = 246
Parent 2: [1, 1, 1, 1, 1, 0, 0, 1] | Fitness = 216
Parent 3: [1, 1, 1, 1, 0, 0, 1, 0] | Fitness = 206
Parent 4: [1, 0, 1, 1, 0, 1, 1, 0] | Fitness = 204
Parent 5: [1, 0, 1, 1, 0, 0, 1, 0] | Fitness = 202
Parent 6: [1, 1, 0, 0, 1, 0, 1, 1] | Fitness = 194
Parent 7: [1, 0, 0, 1, 0, 0, 1, 0] | Fitness = 190
Parent 8: [1, 1, 1, 1, 1, 0, 0, 0] | Fitness = 186
Parent 9: [1, 0, 1, 1, 1, 1, 0, 0] | Fitness = 184
Parent 10: [0, 0, 1, 1, 1, 0, 1, 0] | Fitness = 182
Parent 11: [1, 1, 1, 0, 1, 0, 1, 0] | Fitness = 176
Parent 12: [0, 1, 0, 1, 1, 1, 1, 0] | Fitness = 176
Parent 13: [1, 1, 0, 1, 1, 1, 0, 0] | Fitness = 176
Parent 14: [0, 0, 1, 1, 0, 0, 1, 1] | Fitness = 172
Parent 15: [1, 0, 1, 0, 1, 0, 1, 0] | Fitness = 172
Parent 16: [1, 1, 0, 1, 0, 0, 0, 1] | Fitness = 164
Parent 17: [1, 